# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [8]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

print("Dataframe shape")
print(df.shape[0], df.shape[1])
print("\nColumns :")
print(df.columns.tolist())
print("\nFirst rows :")
print(df.head())


Dataframe shape
10910 26

Columns :
['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value', 'Response', 'Coverage', 'Education', 'Effective To Date', 'EmploymentStatus', 'Gender', 'Income', 'Location Code', 'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size', 'Vehicle Type']

First rows :
   Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   Coverage Education Effective To Date EmploymentSta

In [2]:
# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("\nCleaned columns:")
print(df.columns.tolist())


Cleaned columns:
['unnamed:_0', 'customer', 'state', 'customer_lifetime_value', 'response', 'coverage', 'education', 'effective_to_date', 'employmentstatus', 'gender', 'income', 'location_code', 'marital_status', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception', 'number_of_open_complaints', 'number_of_policies', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size', 'vehicle_type']


In [17]:

# Filter clients low claim + response "Yes"
print("Clients with low claim amount AND response Yes")

df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Check for the presence of necessary columns

print("- total_claim_amount :", "total_claim_amount" in df.columns)
print("- response :", "response" in df.columns)

# Unique values of response
if 'Response' in df.columns:
    df = df.rename(columns={'Response': 'response'})
print("\nValues of 'response' :", df['response'].unique())

# Filter: total_claim_amount < 1000 and response == "Yes"
df_filtered = df[(df['total_claim_amount'] < 1000) & (df['response'] == 'Yes')]

print(f"Clients: {df_filtered.shape[0]}",df_filtered.head())

# Statistics
print(f"\nStatistics")
print(f"Average claim amount : {df_filtered['total_claim_amount'].mean():.2f}$")
print(f"Median claim amount : {df_filtered['total_claim_amount'].median():.2f}$")
if 'customer_lifetime_value' in df_filtered.columns:
    print(f"Average CLV : {df_filtered['customer_lifetime_value'].mean():.2f}$")

Clients with low claim amount AND response Yes
- total_claim_amount : True
- response : True

Values of 'response' : ['No' 'Yes' nan]
Clients: 1399     unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11        

In [24]:
# 2. Analyse by policy_type and gender (response "Yes")
print(" Analysis for clients who responded Yes")


# Only "Yes"
df_yes = df[df['response'] == 'Yes']
print(f"Number of clients with response 'Yes' : {df_yes.shape[0]}")

# mean of monthly_premium by policy_type and gender
print("\nMean of monthly_premium by policy_type and gender")
pivot_premium = pd.pivot_table(df_yes,values='monthly_premium_auto',index='policy_type',columns='gender',aggfunc='mean')
pivot_premium = pivot_premium.round(2)
print(pivot_premium)

# Mean of customer_lifetime_value by policy_type and gender
print("\nMean of customer_lifetime_value by policy_type and gender")
pivot_clv = pd.pivot_table(df_yes,values='customer_lifetime_value',index='policy_type',columns='gender',aggfunc='mean')
pivot_clv = pivot_clv.round(2)
print(pivot_clv)

# Mean of total_claim_amount by policy_type and gender
print("\nMean of total_claim_amount by policy_type and gender")
pivot_claim = pd.pivot_table(df_yes,values='total_claim_amount',index='policy_type',columns='gender',aggfunc='mean')
pivot_claim = pivot_claim.round(2)
print(pivot_claim)

# Number of clients
print("\nNumber of clients by policy_type and gender")
pivot_count = pd.pivot_table(df_yes,values='customer_lifetime_value',index='policy_type',columns='gender',aggfunc='count')
print(pivot_count)

# Profitability
print("\nProfitability")
print("Most profitable segment = High CLV + Low Claims")
print("\nCalculating the CLV / Claims ratio (the higher, the better):")

# Ratio
ratio_profit = pivot_clv / pivot_claim
ratio_profit = ratio_profit.round(2)
print(ratio_profit)

# best sgment
best_segment = ratio_profit.stack().idxmax()
best_ratio = ratio_profit.stack().max()
print(f"\nBest segment:", best_segment[0])
print(f"Gender : best_segment[1]")
print(f"Ratio CLV/Claims: {best_ratio:.2f}")

# Best segment low-risk 
min_claim_segment = pivot_claim.stack().idxmin()
min_claim = pivot_claim.stack().min()
print(f"\nLow-risk segment (lowest claims):")
print(f"Policy Type : {min_claim_segment[0]}")
print(f"Gender : {min_claim_segment[1]}")
print(f"Claim moyen : {min_claim:.2f}$")

 Analysis for clients who responded Yes
Number of clients with response 'Yes' : 1466

Mean of monthly_premium by policy_type and gender
gender              F      M
policy_type                 
Corporate Auto  94.30  92.19
Personal Auto   99.00  91.09
Special Auto    92.31  86.34

Mean of customer_lifetime_value by policy_type and gender
gender                F        M
policy_type                     
Corporate Auto  7712.63  7944.47
Personal Auto   8339.79  7448.38
Special Auto    7691.58  8247.09

Mean of total_claim_amount by policy_type and gender
gender               F       M
policy_type                   
Corporate Auto  433.74  408.58
Personal Auto   452.97  457.01
Special Auto    453.28  429.53

Number of clients by policy_type and gender
gender            F    M
policy_type             
Corporate Auto  169  154
Personal Auto   540  536
Special Auto     35   32

Profitability
Most profitable segment = High CLV + Low Claims

Calculating the CLV / Claims ratio (the higher, the 

In [28]:


# 3: Number of clients / state (filter > 500)



# Filter: only states with > 500 clients
States_more_500 = total_client_state[total_client_state > 500]
print("\nStates with more than 500 clients: ",States_more_500)
print(f"\nNumber of states with > 500 clients : {len(States_more_500)}")


States with more than 500 clients:  state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

Number of states with > 500 clients : 5


In [29]:


# 4: Max, Min, Median CLV (education and gender)
print("Max, Min, Median CLV par Education et Gender")


# Max
print("\nMax CLV")
pivot_max = pd.pivot_table(df,values='customer_lifetime_value',index='education',columns='gender', aggfunc='max')
pivot_max = pivot_max.round(2)
print(pivot_max)

# Min
print("\nMin CLV")
pivot_min = pd.pivot_table(df,values='customer_lifetime_value',index='education',columns='gender',aggfunc='min')
pivot_min = pivot_min.round(2)
print(pivot_min)

# MMedian
print("\nMedian CLV")
pivot_median = pd.pivot_table(df,values='customer_lifetime_value',index='education',columns='gender',aggfunc='median')
pivot_median = pivot_median.round(2)
print(pivot_median)





Max, Min, Median CLV par Education et Gender

Max CLV
gender                       F         M
education                               
Bachelor              73225.96  67907.27
College               61850.19  61134.68
Doctor                44856.11  32677.34
High School or Below  55277.45  83325.38
Master                51016.07  50568.26

Min CLV
gender                      F        M
education                             
Bachelor              1904.00  1898.01
College               1898.68  1918.12
Doctor                2395.57  2267.60
High School or Below  2144.92  1940.98
Master                2417.78  2272.31

Median CLV
gender                      F        M
education                             
Bachelor              5640.51  5548.03
College               5623.61  6005.85
Doctor                5332.46  5577.67
High School or Below  6039.55  6286.73
Master                5729.86  5579.10


In [31]:
# CONCLUSIONS
print("CONCLUSIONS")


print("\n1. Profits :")
print(f"Most profitable segment : {best_segment[0]} + {best_segment[1]}")
print(f"CLV/claims ratio: {best_ratio:.2f}")
print(f"Low-risk segment: {min_claim_segment[0]} + {min_claim_segment[1]}")

print("\n2. Geography :")
print(f"   - {len(etats_plus_500)} States have more than 500 clients")
print(f"   - Main states: {', '.join(etats_plus_500.index[:3].tolist())}")

print("\n3. Education and gender :")
max_clv_pos = pivot_median.stack().idxmax()
max_clv_val = pivot_median.stack().max()
print(f"Median of highter CLV : {max_clv_pos[0]} + {max_clv_pos[1]}")
print(f"Value: {max_clv_val:.2f}$")

min_clv_pos = pivot_median.stack().idxmin()
min_clv_val = pivot_median.stack().min()
print(f"Median of lower CLV: {min_clv_pos[0]} + {min_clv_pos[1]}")
print(f"Value: {min_clv_val:.2f}$")

CONCLUSIONS

1. Profits :
Most profitable segment : Corporate Auto + M
CLV/claims ratio: 19.44
Low-risk segment: Corporate Auto + M

2. Geography :
   - 5 States have more than 500 clients
   - Main states: California, Oregon, Arizona

3. Education and gender :
Median of highter CLV : High School or Below + M
Value: 6286.73$
Median of lower CLV: Doctor + F
Value: 5332.46$
